In [ ]:
import torch
from AHP_Dataset import AHP_Dataset as AHP
from custom_models.SS_v1 import SS_v1
import SupervisedMLFramework as sml
import matplotlib.pyplot as plt
from custom_models.unet_batchnorm import UNet
import time

TRAIN_MAPPING = "data\\AHP\\train_annotations.pkl"
TEST_MAPPING = "data\\AHP\\test_annotations.pkl"
IMG_DIR = "data\\AHP\\AHP\\train\\Processed_Images"
GT_DIR = "data\\AHP\\AHP\\train\\Processed_Annotations"

In [ ]:
file_path = "C:\\Users\\sayba\\Documents\\University\\Spring_2023_T8\\1896\\human_segmentation\\checkpoints\\long_trains\\UNet_Standard_BatchNorm.pt"

model_save = torch.load(file_path)

In [ ]:
model = UNet()
model.load_state_dict(model_save['model_state_dict'])
test_dataset = AHP(TEST_MAPPING, IMG_DIR, GT_DIR)
train_dataset = AHP(TRAIN_MAPPING, IMG_DIR, GT_DIR)

model = sml.SupervisedMLFramework("eval", model, None, None, init_weights=False, batch_size=32)


avg_percent_pixels_correct = 0
for i, (X, y) in enumerate(test_dataset):

    if i==24:
        break

    fig, axs = plt.subplots(1,4)
    fig.set_size_inches(15,20)

    axs[0].imshow(X.permute(1,2,0))
    axs[0].set_xlabel("Original Input")

    axs[1].imshow(y)
    axs[1].set_xlabel("Original Ground Truth")


    X = torch.unsqueeze(X, dim=0)
    y = torch.unsqueeze(y, dim=0)

    now = time.time()

    pred = model.predict(X)

    print(f"prediction time: {time.time() - now}")
    axs[2].imshow(pred)
    axs[2].set_xlabel("Predicted Classes After Training")

    axs[3].imshow(torch.squeeze(torch.unsqueeze(pred, dim=0) == y))
    axs[3].set_xlabel("Correctly predicted pixels")
    plt.show()
    #print(torch.unsqueeze(pred, dim=0).shape)
    # print(y.shape)
    avg_percent_pixels_correct += torch.sum(torch.unsqueeze(pred, dim=0) == y) / y.shape[1]**2

print(avg_percent_pixels_correct / 24)
#print(avg_percent_pixels_correct)
#print(pred.shape)
# print(pred.shape)
# print(y.shape)